# **PROBLEMAS TIPO FLOW-SHOP**

Se analiza el problema llevando a cabo la programación del material obtenido.

In [ ]:
import numpy as np

#tiempos de procesamiento (Tij) para las tareas y máquinas
processing_times = np.array([
    [5, 5, 3, 6, 3], #M1
    [4, 4, 2, 4, 4], #M2
    [4, 4, 2, 4, 1], #M3
    [3, 6, 3, 2, 5]  #M4
])

n_machines = processing_times.shape[0]  #tiempo de procesamiento para maquinas
n_tasks = processing_times.shape[1]     #para tareas

#Inicializamos el makespan
makespan = np.zeros((n_machines, n_tasks))

#se inicia el calculo del makespan
makespan[0, 0] = processing_times[0, 0]

for j in range(1, n_tasks):
    makespan[0, j] = makespan[0, j-1] + processing_times[0, j]

for i in range(1, n_machines):
    makespan[i, 0] = makespan[i-1, 0] + processing_times[i, 0]

for i in range(1, n_machines):
    for j in range(1, n_tasks):
        makespan[i, j] = max(makespan[i-1, j], makespan[i, j-1]) + processing_times[i, j]

print("Matriz de Makespan (Cij):")
print(makespan)

final_makespan = makespan[-1, -1]
print(f"El Makespan final es: {final_makespan}")


Matriz de Makespan (Cij):
[[ 5. 10. 13. 19. 22.]
 [ 9. 14. 16. 23. 27.]
 [13. 18. 20. 27. 28.]
 [16. 24. 27. 29. 34.]]
El Makespan final es: 34.0


# **Implementación con mutación basada en desplazamiento**

In [ ]:
import random

#fitness
def evaluate(chromosome, tiempos):
    n_tareas = len(chromosome)
    m_maquinas = len(tiempos)

#Matriz de tiempos
    C = [[0] * (n_tareas + 1) for _ in range(m_maquinas)]

#Calcular el makespan
    for i in range(1, n_tareas + 1):
        for j in range(1, m_maquinas + 1):
            if i == 1 and j == 1:
                C[j-1][i] = tiempos[j-1][chromosome[i-1]-1]
            elif i == 1:
                C[j-1][i] = C[j-2][i] + tiempos[j-1][chromosome[i-1]-1]
            elif j == 1:
                C[j-1][i] = C[j-1][i-1] + tiempos[j-1][chromosome[i-1]-1]
            else:
                C[j-1][i] = max(C[j-1][i-1], C[j-2][i]) + tiempos[j-1][chromosome[i-1]-1]

    makespan = C[m_maquinas-1][n_tareas]
    return makespan

def mutacion_desplazamiento(chromosome, tiempos):
    best_chromosome = chromosome[:]
    best_score = evaluate(chromosome, tiempos)
    print(f"Camino inicial: {chromosome} con Makespan: {best_score}")

    for _ in range(10):
        new_chromosome = chromosome[:]
        i = random.randint(0, len(new_chromosome) - 1)
        j = random.randint(0, len(new_chromosome) - 1)

        gen = new_chromosome.pop(i)
        new_chromosome.insert(j, gen)

        score = evaluate(new_chromosome, tiempos)
        print(f"Nuevo camino: {new_chromosome} con Makespan: {score}")

        if score < best_score:
            best_chromosome = new_chromosome[:]
            best_score = score

    print(f"Mejor camino encontrado: {best_chromosome} con Makespan: {best_score}")
    return best_chromosome

chromosome = [1, 2, 3, 4, 5]
tiempos = [
    [5, 5, 3, 6, 3],
    [4, 4, 2, 4, 4],
    [4, 4, 2, 4, 1],
    [3, 6, 3, 2, 5]
]

mutacion_desplazamiento(chromosome, tiempos)


Camino inicial: [1, 2, 3, 4, 5] con Makespan: 34
Nuevo camino: [1, 4, 2, 3, 5] con Makespan: 38
Nuevo camino: [1, 5, 2, 3, 4] con Makespan: 32
Nuevo camino: [1, 3, 2, 4, 5] con Makespan: 34
Nuevo camino: [1, 2, 4, 5, 3] con Makespan: 34
Nuevo camino: [4, 1, 2, 3, 5] con Makespan: 38
Nuevo camino: [2, 3, 4, 1, 5] con Makespan: 35
Nuevo camino: [1, 2, 3, 4, 5] con Makespan: 34
Nuevo camino: [1, 5, 2, 3, 4] con Makespan: 32
Nuevo camino: [1, 3, 2, 4, 5] con Makespan: 34
Nuevo camino: [2, 3, 4, 5, 1] con Makespan: 33
Mejor camino encontrado: [1, 5, 2, 3, 4] con Makespan: 32


[1, 5, 2, 3, 4]

Evaluamos multiples camino, este metodo nos parece realmente ideal dado que su concepto se ajusta directamente a los requerimientos de optimización y se liga al contexto.

No obstante, dado que analizamos la teoria y releemos el contexto del ejercicio, nos dimos cuenta que esta tecnica encuentra una solución local, con lo que nos preguntamos:

**¿Qué pasaria si evaluamos una solución mucho mas global?**

# **Función cruce desplazamiento : solución global**

In [ ]:
#Cruce basado en desplazamiento con makespan
def cruce_desplazamiento(padre, madre, tiempos):

    i = random.randint(0, len(padre) - 1)
    j = random.randint(i + 1, len(padre))

    #subcadena padre
    subcadena = padre[i:j]
    resto = [gen for gen in madre if gen not in subcadena]

    #insertamos la subcadena
    posicion_insercion = random.randint(0, len(resto))
    hijo = resto[:posicion_insercion] + subcadena + resto[posicion_insercion:]

    makespan_hijo = evaluate(hijo, tiempos)

    print(f"Cruce realizado: Padre: {padre}, Madre: {madre}, Hijo: {hijo}, Makespan del hijo: {makespan_hijo}")
    return hijo, makespan_hijo

padre = [1, 2, 3, 4, 5]
madre = [5, 4, 3, 2, 1]
tiempos = [
    [5, 5, 3, 6, 3],
    [4, 4, 2, 4, 4],
    [4, 4, 2, 4, 1],
    [3, 6, 3, 2, 5]
]

hijo, makespan_hijo = cruce_desplazamiento(padre, madre, tiempos)


Cruce realizado: Padre: [1, 2, 3, 4, 5], Madre: [5, 4, 3, 2, 1], Hijo: [4, 5, 3, 2, 1], Makespan del hijo: 34


In [ ]:
import random

#fitness
def evaluate(chromosome, tiempos):
    n_tareas = len(chromosome)
    m_maquinas = len(tiempos)

    C = [[0] * (n_tareas + 1) for _ in range(m_maquinas)]

    for i in range(1, n_tareas + 1):
        for j in range(1, m_maquinas + 1):
            if i == 1 and j == 1:
                C[j-1][i] = tiempos[j-1][chromosome[i-1]-1]
            elif i == 1:
                C[j-1][i] = C[j-2][i] + tiempos[j-1][chromosome[i-1]-1]
            elif j == 1:
                C[j-1][i] = C[j-1][i-1] + tiempos[j-1][chromosome[i-1]-1]
            else:
                C[j-1][i] = max(C[j-1][i-1], C[j-2][i]) + tiempos[j-1][chromosome[i-1]-1]

    makespan = C[m_maquinas-1][n_tareas]
    return makespan

def mutacion_desplazamiento(chromosome, tiempos):
    best_chromosome = chromosome[:]
    best_score = evaluate(chromosome, tiempos)
    print(f"Camino inicial: {chromosome} con Makespan: {best_score}")

    for _ in range(100):
        new_chromosome = chromosome[:]
        i = random.randint(0, len(new_chromosome) - 1)
        j = random.randint(0, len(new_chromosome) - 1)

        gen = new_chromosome.pop(i)
        new_chromosome.insert(j, gen)

        score = evaluate(new_chromosome, tiempos)
        print(f"Nuevo camino: {new_chromosome} con Makespan: {score}")

        if score < best_score:
            best_chromosome = new_chromosome[:]
            best_score = score

    print(f"Mejor camino encontrado: {best_chromosome} con Makespan: {best_score}")
    return best_chromosome

chromosome = [1, 2, 3, 4, 5]
tiempos = [
    [5, 5, 3, 6, 3],
    [4, 4, 2, 4, 4],
    [4, 4, 2, 4, 1],
    [3, 6, 3, 2, 5]
]

mutacion_desplazamiento(chromosome, tiempos)


Camino inicial: [1, 2, 3, 4, 5] con Makespan: 34
Nuevo camino: [2, 3, 4, 1, 5] con Makespan: 35
Nuevo camino: [1, 2, 3, 5, 4] con Makespan: 34
Nuevo camino: [1, 2, 3, 4, 5] con Makespan: 34
Nuevo camino: [1, 3, 2, 4, 5] con Makespan: 34
Nuevo camino: [1, 2, 3, 5, 4] con Makespan: 34
Nuevo camino: [1, 3, 2, 4, 5] con Makespan: 34
Nuevo camino: [3, 1, 2, 4, 5] con Makespan: 34
Nuevo camino: [1, 2, 4, 5, 3] con Makespan: 34
Nuevo camino: [4, 1, 2, 3, 5] con Makespan: 38
Nuevo camino: [1, 2, 3, 4, 5] con Makespan: 34
Nuevo camino: [1, 3, 4, 2, 5] con Makespan: 38
Nuevo camino: [3, 1, 2, 4, 5] con Makespan: 34
Nuevo camino: [1, 3, 2, 4, 5] con Makespan: 34
Nuevo camino: [3, 1, 2, 4, 5] con Makespan: 34
Nuevo camino: [1, 3, 4, 5, 2] con Makespan: 36
Nuevo camino: [2, 3, 1, 4, 5] con Makespan: 34
Nuevo camino: [2, 3, 1, 4, 5] con Makespan: 34
Nuevo camino: [5, 1, 2, 3, 4] con Makespan: 32
Nuevo camino: [2, 1, 3, 4, 5] con Makespan: 34
Nuevo camino: [1, 2, 4, 3, 5] con Makespan: 34
Nuevo camin

[5, 1, 2, 3, 4]

# **Optimización de solución global:**

In [ ]:
#Cruce basado en desplazamiento con makespan
def cruce_desplazamiento(padre, madre, tiempos):

    i = random.randint(0, len(padre) - 1)
    j = random.randint(i + 1, len(padre))

    #subcadena padre
    subcadena = padre[i:j]
    resto = [gen for gen in madre if gen not in subcadena]

    #insertamos la subcadena
    posicion_insercion = random.randint(0, len(resto))
    hijo = resto[:posicion_insercion] + subcadena + resto[posicion_insercion:]

    makespan_hijo = evaluate(hijo, tiempos)

    print(f"Cruce realizado: Padre: {padre}, Madre: {madre}, Hijo: {hijo}, Makespan del hijo: {makespan_hijo}")
    return hijo, makespan_hijo
#Ahora hemos usado los resultados de la solución local en la global.
padre = [1, 5, 2, 3, 4]
madre = [5, 1, 2, 3, 4]
tiempos = [
    [5, 5, 3, 6, 3],
    [4, 4, 2, 4, 4],
    [4, 4, 2, 4, 1],
    [3, 6, 3, 2, 5]
]

hijo, makespan_hijo = cruce_desplazamiento(padre, madre, tiempos)


Cruce realizado: Padre: [1, 5, 2, 3, 4], Madre: [5, 1, 2, 3, 4], Hijo: [5, 1, 2, 4, 3], Makespan del hijo: 32


implementar algoritmo genetico, para saber cuántas iteraciones, usar tools y deep con las funciones implementadas. registrar en el tools los operadores e invocar, para evaluación, cruce y mutación.

In [ ]:
!pip install deap

In [ ]:
import random
from deap import base, creator, tools, algorithms


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


def evaluate(individual, tiempos):
    n = len(individual)
    m = len(tiempos)
    tiempo_fin = [[0] * n for _ in range(m)]

    for j in range(n):
        for i in range(m):
            if j == 0 and i == 0:
                tiempo_fin[i][j] = tiempos[i][individual[j] - 1]
            elif i == 0:
                tiempo_fin[i][j] = tiempo_fin[i][j - 1] + tiempos[i][individual[j] - 1]
            elif j == 0:
                tiempo_fin[i][j] = tiempo_fin[i - 1][j] + tiempos[i][individual[j] - 1]
            else:
                tiempo_fin[i][j] = max(tiempo_fin[i - 1][j], tiempo_fin[i][j - 1]) + tiempos[i][individual[j] - 1]

    return tiempo_fin[m - 1][n - 1],

def cruce_desplazamiento(padre, madre):
    i = random.randint(0, len(padre) - 1)
    j = random.randint(i + 1, len(padre))


    subcadena1 = padre[i:j]
    resto1 = [gen for gen in madre if gen not in subcadena1]

    posicion_insercion1 = random.randint(0, len(resto1))
    hijo1 = resto1[:posicion_insercion1] + subcadena1 + resto1[posicion_insercion1:]

    subcadena2 = madre[i:j]
    resto2 = [gen for gen in padre if gen not in subcadena2]

    posicion_insercion2 = random.randint(0, len(resto2))
    hijo2 = resto2[:posicion_insercion2] + subcadena2 + resto2[posicion_insercion2:]

    hijo1 = creator.Individual(hijo1)
    hijo2 = creator.Individual(hijo2)

    return hijo1, hijo2

def mutacion_desplazamiento(individuo):
    i = random.randint(0, len(individuo) - 1)
    j = random.randint(0, len(individuo) - 1)

    gen = individuo.pop(i)
    individuo.insert(j, gen)

    return individuo,

#creamos el toolbox
toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(1, 6), 5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate, tiempos=[
    [5, 5, 3, 6, 3],
    [4, 4, 2, 4, 4],
    [4, 4, 2, 4, 1],
    [3, 6, 3, 2, 5]
])

toolbox.register("mate", cruce_desplazamiento)
toolbox.register("mutate", mutacion_desplazamiento)
toolbox.register("select", tools.selTournament, tournsize=3)


def algoritmo_genetico():
    population = toolbox.population(n=100)
    cxpb, mutpb, ngen = 0.5, 0.2, 40
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("min", lambda x: min(f[0] for f in x))
    stats.register("max", lambda x: max(f[0] for f in x))
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + stats.fields

    population, log = algorithms.eaSimple(
        population, toolbox, cxpb, mutpb, ngen, stats=stats, verbose=True
    )

    best_individual = tools.selBest(population, 1)[0]
    print(f"\nMejor camino encontrado: {best_individual}")
    print(f"Makespan: {best_individual.fitness.values[0]}")

    for record in log:
        print(record)

algoritmo_genetico()


gen	nevals	avg  	min	max
0  	100   	34.68	32 	38 
1  	69    	33.57	32 	38 
2  	63    	32.86	32 	36 
3  	74    	33.03	32 	38 
4  	50    	32.55	32 	38 
5  	63    	32.78	32 	38 
6  	58    	32.94	32 	36 
7  	54    	32.55	32 	36 
8  	55    	32.74	32 	36 
9  	63    	32.56	32 	38 
10 	51    	32.74	32 	38 
11 	52    	32.55	32 	38 
12 	59    	32.96	32 	38 
13 	67    	32.82	32 	36 
14 	60    	32.59	32 	36 
15 	62    	32.81	32 	38 
16 	64    	32.91	32 	38 
17 	60    	32.71	32 	36 
18 	55    	32.54	32 	36 
19 	63    	32.79	32 	36 
20 	52    	32.94	32 	38 
21 	57    	32.8 	32 	38 
22 	60    	32.68	32 	36 
23 	73    	32.78	32 	38 
24 	60    	32.56	32 	36 
25 	73    	32.89	32 	36 
26 	57    	32.65	32 	36 
27 	61    	32.58	32 	36 
28 	47    	32.66	32 	38 
29 	61    	32.78	32 	38 
30 	62    	32.81	32 	36 
31 	60    	32.56	32 	36 
32 	59    	32.76	32 	38 
33 	65    	32.59	32 	38 
34 	60    	32.87	32 	38 
35 	51    	32.55	32 	36 
36 	68    	32.98	32 	38 
37 	66    	32.6 	32 	36 
38 	65    	32.5 	32 	36 


#**Conclusiones**



*   Mutación basada en desplazamiento como método efectivo.
*   Cruce basado en desplazamiento cuenta con mayor aleatoriedad.
*   Análisis de exploración  vs explotación.
*   Diversidad en el espacio de búsqueda.
*   El rol del makespan fue clave para la evaluación.
*   La mutación es mas efectiva para  Problema Flow-Shop.









